In [34]:
import psycopg2
import psycopg2.extras as extras
import pandas as pd
import json
from datetime import datetime

from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest
from google.oauth2 import service_account


In [35]:
projectId='pongthorn'
table_id = f"{projectId}.DemoSMartDW.incident"

client = bigquery.Client(project=projectId)

In [36]:
try:
    client.get_table(table_id)  # Make an API request.
    print("Table {} already exists.".format(table_id))
except NotFound:
    raise Exception("Table {} is not found.".format(table_id))

Table pongthorn.DemoSMartDW.incident already exists.


In [37]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
         database='SMartDB', user='postgres',
      password='P@ssw0rd', host='localhost', 
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

In [38]:
sql_incident="""

select
incident.id as id, incident.incident_no as incident_no,

severity.id as  severity_id,
severity.severity_name as  severity_name,

service_level.sla_name as sla,

product_type.productype_name as product_type,brand.brand_name as brand,
xtype.incident_type_name as incident_type,
status.incident_status_name as status,
service.service_type_name service_type

,(select count(*) from  app_incident_detail  as detail where  detail.incident_master_id=incident.id ) as count_detail

,TO_CHAR(incident.incident_datetime  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as open_datetime
,TO_CHAR(incident.incident_close_datetime  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as close_datetime

,TO_CHAR(incident.incident_problem_start  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as response_datetime
,TO_CHAR(incident.incident_problem_end  AT TIME ZONE 'Asia/Bangkok','YYYY-MM-DD HH24:MI') as resolved_datetime

,TO_CHAR(incident.updated_at,'YYYY-MM-DD HH24:MI:SS') as updated_at


from app_incident as incident
inner join app_incident_type as  xtype on incident.incident_type_id = xtype.id

inner join  app_incident_status as status on incident.incident_status_id = status.id
inner join  app_incident_severity as severity on  incident.incident_severity_id = severity.id
inner join  app_service_type as service on incident.service_type_id= service.id

inner join app_inventory as inventory on incident.inventory_id = inventory.id

inner join app_brand as brand on inventory.brand_id = brand.id

inner join app_product_type as product_type on inventory.product_type_id = product_type.id
inner join app_sla as service_level on inventory.customer_sla_id = service_level.id
where incident.incident_status_id =4  and incident.updated_at>=%(start_date_param)s

order by incident.incident_datetime desc


"""

In [39]:
start_date_query='2000-01-01'
dict_params={"start_date_param":start_date_query}
df_all=list_data(sql_incident,dict_params,get_postgres_conn())

if df_all.empty==True:
    print("no transsaction update")
    exit()

In [40]:
print(df_all.info())
df_all.tail(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2418 entries, 0 to 2417
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 2418 non-null   int64 
 1   incident_no        2418 non-null   object
 2   severity_id        2418 non-null   int64 
 3   severity_name      2418 non-null   object
 4   sla                2418 non-null   object
 5   product_type       2418 non-null   object
 6   brand              2418 non-null   object
 7   incident_type      2418 non-null   object
 8   status             2418 non-null   object
 9   service_type       2418 non-null   object
 10  count_detail       2418 non-null   int64 
 11  open_datetime      2418 non-null   object
 12  close_datetime     2418 non-null   object
 13  response_datetime  2418 non-null   object
 14  resolved_datetime  2418 non-null   object
 15  updated_at         2418 non-null   object
dtypes: int64(3), object(13)
memory usage: 302.

,id,incident_no,severity_id,severity_name,sla,product_type,brand,incident_type,status,service_type,count_detail,open_datetime,close_datetime,response_datetime,resolved_datetime,updated_at
2413,573,SR-ES-22-0573,2,Major,24x7 4Hrs Response Time,Storage,NetApp,General Incident,Closed,Incident,3,2020-05-14 13:48,2020-05-29 19:07,2020-05-14 16:26,2020-05-21 17:27,2022-05-06 15:31:10
2414,572,SR-ES-22-0572,2,Major,24x7 4Hrs Response Time,Storage,NetApp,General Incident,Closed,Incident,2,2020-04-23 14:58,2020-04-24 11:08,2020-04-23 15:31,2020-04-23 20:03,2022-05-06 14:43:33
2415,571,SR-ES-22-0571,2,Major,24x7 4Hrs Response Time,Storage,NetApp,General Incident,Closed,Incident,1,2020-04-16 16:26,2020-04-17 16:13,2020-04-17 16:13,2020-04-17 16:13,2022-05-06 14:27:32
2416,570,SR-ES-22-0570,2,Major,24x7 4Hrs Response Time,Storage,NetApp,General Incident,Closed,Incident,2,2020-03-15 13:36,2020-03-16 20:45,2020-03-15 15:03,2020-03-16 19:31,2022-05-06 14:16:59
2417,569,SR-ES-22-0569,2,Major,24x7 4Hrs Response Time,Software,NetApp,Power Supply Failure,Closed,Incident,1,2019-11-01 08:30,2019-11-01 15:27,2019-11-01 13:38,2019-11-01 15:02,2022-05-06 13:58:59


In [41]:
# convert object to datetime
dateTimeCols=['open_datetime','response_datetime','resolved_datetime','close_datetime','updated_at']
for col in dateTimeCols:
 df_all[col]=pd.to_datetime(df_all[col], format='%Y-%m-%d %H:%M',errors= 'coerce')



In [42]:
dt_imported=datetime.now()
str_imported=dt_imported.strftime('%Y-%m-%d %H:%M:%S')
print(f"Imported DateTime: {str_imported}" )
df_all['imported_at']=dt_imported

Imported DateTime: 2023-04-03 14:25:56


In [43]:
df_all.dropna(inplace=True)

print(df_all.info())
df_all.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2418 entries, 0 to 2417
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 2418 non-null   int64         
 1   incident_no        2418 non-null   object        
 2   severity_id        2418 non-null   int64         
 3   severity_name      2418 non-null   object        
 4   sla                2418 non-null   object        
 5   product_type       2418 non-null   object        
 6   brand              2418 non-null   object        
 7   incident_type      2418 non-null   object        
 8   status             2418 non-null   object        
 9   service_type       2418 non-null   object        
 10  count_detail       2418 non-null   int64         
 11  open_datetime      2418 non-null   datetime64[ns]
 12  close_datetime     2418 non-null   datetime64[ns]
 13  response_datetime  2418 non-null   datetime64[ns]
 14  resolved

,id,incident_no,severity_id,severity_name,sla,product_type,brand,incident_type,status,service_type,count_detail,open_datetime,close_datetime,response_datetime,resolved_datetime,updated_at,imported_at
0,2582,SR-ES-23-2582,3,Minor,24x7 4Hrs Response Time,Storage,NetApp,Hard Disk Drive Failure,Closed,Incident,2,2023-04-03 08:20:00,2023-04-03 12:00:00,2023-04-03 08:20:00,2023-04-03 12:00:00,2023-04-03 13:39:36,2023-04-03 14:25:56.410999
1,2581,SR-ES-23-2581,3,Minor,24x7 4Hrs Resolution Time,Storage,NetApp,Hard Disk Drive Failure,Closed,Incident,2,2023-04-02 23:20:00,2023-04-03 12:00:00,2023-04-02 23:20:00,2023-04-03 12:00:00,2023-04-03 13:28:19,2023-04-03 14:25:56.410999
2,2576,SR-ES-23-2576,3,Minor,24x7 4Hrs Resolution Time,Storage,NetApp,Hard Disk Drive Failure,Closed,Incident,2,2023-03-31 21:00:00,2023-04-02 12:00:00,2023-03-31 21:00:00,2023-04-02 12:00:00,2023-04-02 21:53:20,2023-04-03 14:25:56.410999
3,2575,SR-ES-23-2575,2,Major,24x7 6Hrs Resolution Time,Software,YIP,Software,Closed,Incident,1,2023-03-31 12:00:00,2023-04-01 23:30:00,2023-03-31 13:30:00,2023-04-01 23:00:00,2023-04-01 23:57:38,2023-04-03 14:25:56.410999
4,2574,SR-ES-23-2574,3,Minor,24x7 4Hrs Response Time,Software,VMWare,General Incident,Closed,Incident,2,2023-03-30 19:54:00,2023-03-31 15:06:00,2023-03-30 20:06:00,2023-03-31 15:06:00,2023-04-03 12:00:09,2023-04-03 14:25:56.410999


In [44]:
def insertDataFrameToBQ(df):
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_APPEND",
        )

        job = client.load_table_from_dataframe(
            df, table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total ", len(df), "Imported closed incident to bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        for e in job.errors:
            print('ERROR: {}'.format(e['message']))

try:
    insertDataFrameToBQ(df_all)
except Exception as ex:
    raise ex


Total  2418 Imported closed incident to bigquery successfully


In [45]:
def load_data_bq(sql:str):
 query_result=client.query(sql)
 dfx=query_result.to_dataframe()
 return dfx

sql_query=f""" SELECT * FROM `{table_id}`  LIMIT 10 """
df=load_data_bq(sql_query)
df


,id,incident_no,severity_id,severity_name,sla,product_type,brand,incident_type,status,service_type,count_detail,open_datetime,close_datetime,response_datetime,resolved_datetime,updated_at,imported_at
0,206,SR-ES-22-0206,1,Critical,24x7 4Hrs Resolution Time,Server,HPE,Hard Disk Drive Failure,Closed,Incident,1,2022-04-04 11:24:00,2022-04-04 16:00:00,2022-04-04 11:30:00,2022-04-04 16:00:00,2022-04-05 10:13:38,2023-04-03 14:25:56.410999
1,325,SR-ES-22-0325,1,Critical,24x7 4Hrs Resolution Time,Server,HPE,Hard Disk Drive Failure,Closed,Incident,1,2022-04-11 12:00:00,2022-04-11 22:00:00,2022-04-11 12:08:00,2022-04-11 22:00:00,2022-04-12 07:15:16,2023-04-03 14:25:56.410999
2,239,SR-ES-22-0239,1,Critical,24x7 4Hrs Resolution Time,Server,HPE,System Board Failure,Closed,Incident,1,2022-02-26 07:56:00,2022-02-26 12:00:00,2022-02-26 08:00:00,2022-02-26 12:00:00,2022-04-04 21:39:48,2023-04-03 14:25:56.410999
3,24,SR-ES-22-0024,1,Critical,24x7 4Hrs Resolution Time,Server,HPE,Hard Disk Drive Failure,Closed,Incident,1,2022-03-24 17:14:00,2022-03-24 22:00:00,2022-03-24 17:18:00,2022-03-24 22:00:00,2022-03-24 22:08:41,2023-04-03 14:25:56.410999
4,188,SR-ES-22-0188,1,Critical,24x7 4Hrs Resolution Time,Storage,HPE,General Incident,Closed,Incident,0,2022-03-15 10:26:00,2022-03-15 12:00:00,2022-03-15 10:30:00,2022-03-15 12:00:00,2022-04-04 13:38:00,2023-04-03 14:25:56.410999
5,174,SR-ES-22-0174,1,Critical,24x7 4Hrs Resolution Time,Storage,HPE,Hard Disk Drive Failure,Closed,Incident,0,2022-02-17 11:53:00,2022-02-17 17:00:00,2022-02-17 11:59:00,2022-02-17 17:00:00,2022-04-04 12:37:09,2023-04-03 14:25:56.410999
6,173,SR-ES-22-0173,1,Critical,24x7 4Hrs Resolution Time,Server,HPE,Power Supply Failure,Closed,Incident,0,2022-02-15 09:34:00,2022-02-15 14:28:00,2022-02-15 09:40:00,2022-02-15 14:00:00,2022-04-04 12:30:18,2023-04-03 14:25:56.410999
7,326,SR-ES-22-0326,1,Critical,24x7 4Hrs Resolution Time,Server,HPE,Hard Disk Drive Failure,Closed,Incident,1,2022-04-11 12:01:00,2022-04-11 22:00:00,2022-04-11 12:09:00,2022-04-11 22:00:00,2022-04-12 07:21:33,2023-04-03 14:25:56.410999
8,164,SR-ES-22-0164,1,Critical,24x7 4Hrs Resolution Time,Server,HPE,Hard Disk Drive Failure,Closed,Incident,0,2021-12-30 10:51:00,2021-12-30 16:00:00,2021-12-30 10:59:00,2021-12-30 16:00:00,2022-04-04 11:27:22,2023-04-03 14:25:56.410999
9,400,SR-ES-22-0400,1,Critical,24x7 4Hrs Resolution Time,Server,HPE,General Incident,Closed,Incident,1,2022-04-18 11:59:00,2022-04-18 18:00:00,2022-04-18 12:05:00,2022-04-18 18:00:00,2022-04-19 07:45:08,2023-04-03 14:25:56.410999
